In [1]:
%use adventOfCode

import com.toldoven.aoc.notebook.AocClient
val aoc = AocClient.fromFile().interactiveDay(2024, 14)

In [2]:
val testInput = """
        p=0,4 v=3,-3
        p=6,3 v=-1,-3
        p=10,3 v=-1,2
        p=2,0 v=2,-1
        p=0,0 v=1,3
        p=3,0 v=-2,-2
        p=7,6 v=-1,-3
        p=3,0 v=-1,-2
        p=9,3 v=2,3
        p=7,3 v=-1,2
        p=2,4 v=2,-3
        p=9,5 v=-3,-3
        """.trimIndent().lines()

In [3]:
val testRangeX = 0..<11
val testRangeY = 0..<7

val rangeX = 0..<101
val rangeY = 0..<103

data class Space(val xRange: IntRange, val yRange: IntRange) {
    val sizeX = xRange.last - xRange.first + 1
    val sizeY = yRange.last - yRange.first + 1

    infix operator fun contains(p: Vec2) = p.x in xRange && p.y in yRange
}

data class Vec2(val x: Int, val y: Int) {
    infix operator fun plus(o: Vec2) = Vec2(x+o.x, y+o.y)
    infix operator fun minus(o: Vec2) = Vec2(x-o.x, y-o.y)
    infix operator fun times(i: Int) = Vec2(i*x, i*y)
}

val regex = """\bp=(?<px>\d+),(?<py>\d+) v=(?<vx>-?\d+),(?<vy>-?\d+)\b""".toRegex()

fun List<String>.parse() = map {
    val (px, py, vx, vy) = regex.find(it)?.destructured ?: error("No match")
    Pair(Vec2(px.toInt(),py.toInt()), Vec2(vx.toInt(),vy.toInt()))
}

In [19]:
typealias Robot = Pair<Vec2, Vec2>

fun Robot.afterSecondsInSpace(k: Int, s: Space): Vec2 {
    val p = this.first
    val v = this.second

    val px = ((p.x + k*v.x)%s.sizeX + s.sizeX)%s.sizeX
    val py = ((p.y + k*v.y)%s.sizeY + s.sizeY)%s.sizeY
    return Vec2(px, py)
}

fun List<Robot>.afterSecondsInSpace(seconds: Int, space: Space) = map {
    Pair(it.afterSecondsInSpace(seconds, space), it.second)
}

fun Space.quadrants() = listOf (
    Space(0..<this.sizeX/2, 0..<this.sizeY/2), // Top Left
    Space(this.sizeX/2+1..this.xRange.last, 0..<this.sizeY/2), // Top Right
    Space(0..<this.sizeX/2, this.sizeY/2+1..this.yRange.last), // Bottom Left
    Space(this.sizeX/2+1..this.xRange.last, this.sizeY/2+1..this.yRange.last) // Bottom Right
)

fun List<Robot>.safetyFactor(s: Space): Int = filter { (pos, _) -> s.quadrants().any { pos in it } }.groupBy { (pos, _) -> s.quadrants().first { pos in it } }.map { (_, list) -> list.size }.reduce { a, b -> a * b }

In [5]:
val test = testInput.parse()
val testSpace = Space(testRangeX, testRangeY)
val testUpdate = test.afterSecondsInSpace(100, testSpace)

In [6]:
testUpdate.safetyFactor(testSpace)

12

In [7]:
val space = Space(rangeX, rangeY)
val robots = aoc.input().lines().parse()
val update = robots.afterSecondsInSpace(100, space)
val part1 = update.safetyFactor(space)

In [8]:
aoc.submitPartOne(part1)

The answer is correct! Your answer: 224969976 You already completed this part.

In [20]:
fun List<Robot>.updateInSpaceUntil(space: Space, predicate: (List<Robot>) -> Boolean): Int {
    var update = this.afterSecondsInSpace(1, space)
    var seconds = 1
    while (!predicate(update)) {
        update = update.afterSecondsInSpace(1, space)
        seconds++
    }
    return seconds
}

In [21]:
fun List<Robot>.isChristmasTree(): Boolean =
    distinctBy { (pos, _) -> pos.x to pos.y }.size == size

In [23]:
val part2 = robots.updateInSpaceUntil(space) { it.isChristmasTree() }
part2

7892

In [14]:
aoc.submitPartTwo(part2)

The answer is correct! Your answer: 7892 You already completed this part.

In [15]:
val tree = robots.afterSecondsInSpace(part2, space)

